In [1]:
import sys
sys.path.append('../../src/')

In [2]:
%load_ext autoreload
%autoreload 2

from data import load_data
from plotting import plot_data_loader

In [3]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [4]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [5]:
data_dir = "../../data"
images_dir = "../../data/covid_pneumonia"

In [30]:
num_workers = 0
batch_size = 64
image_size=299

In [31]:
aug = T.Compose([T.Resize(size=(image_size, image_size)),
                 T.ToTensor(),
                 T.Normalize(mean = (0, 0, 0),
                              std  = (1, 1, 1))])

train_dataset = datasets.ImageFolder(root=os.path.join(images_dir, "train"),
                                     transform=aug)
val_dataset = datasets.ImageFolder(root=os.path.join(images_dir, "val"),
                                   transform=aug)
test_dataset = datasets.ImageFolder(root=os.path.join(images_dir, "test"),
                                    transform=aug)

data_loader_train = td.DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  drop_last=False,
                                  num_workers=num_workers,
                                  pin_memory=True)
data_loader_val = td.DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                drop_last=False,
                                num_workers=num_workers,
                                pin_memory=True)
data_loader_test = td.DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 drop_last=False,
                                 num_workers=num_workers,
                                 pin_memory=True)

In [32]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_train):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|████████████████████████████████████████████████████████████████████████████████| 166/166 [00:31<00:00,  5.21it/s]


In [34]:
count = len(data_loader_train.dataset) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.5159, 0.5159, 0.5159])
std:  tensor([0.2554, 0.2554, 0.2554])


In [35]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_val):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:06<00:00,  5.35it/s]


In [36]:
count = len(data_loader_val.dataset) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.5135, 0.5135, 0.5135])
std:  tensor([0.2542, 0.2542, 0.2542])


In [37]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_test):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:06<00:00,  5.44it/s]


In [38]:
count = len(data_loader_test.dataset) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.5157, 0.5157, 0.5157])
std:  tensor([0.2546, 0.2546, 0.2546])
